# Building Vision Input from Existing Components

We have most of the code for processing vision from other projects. We just need to bring it together to make a video sensor.

One thing we need to do is quickly process video input to convert to a 1D numpy array. 

Options:
* Split visual field.
* Lightness / colour (start with L).
* Retinotopic mappings.
* Area of interest.

We'll look at square 2D blocks of sqrt(vec_len). Hence, vec_len is recommended to be a power of 4 - 4, 16 or 64. 4 and 16 are preferable to start.

We have a ```VideoSource``` object. We might need to add a BGR to YUV step -  img_out = cv2.cvtColor(img_in, cv2.COLOR_BGR2YUV). In code I set the property 16 to 0 ```self.cam.set(16, 0)```. Ah - 16 indicates whether the image should be converted to RGB. 0 turns this off to get original YUV.

In [ ]:
def tile_array(a, b0, b1):
    """Tile array a in blocks of size b0 * b1"""
    r, c = a.shape                                    # number of rows/columns
    rs, cs = a.strides                                # row/column strides
    x = as_strided(a, (r, b0, c, b1), (rs, 0, cs, 0)) # view as 4D array
    return x.reshape(r*b0, c*b1)  